In [15]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.circuit.library import RGQFTMultiplier

def multiplication(a, b):
    n = max(len(bin(a)), len(bin(b))) - 2

    q = QuantumRegister(4*n, 'q')
    c = ClassicalRegister(2*n, 'c')

    circuit = QuantumCircuit(q, c)

    
    a_binary = format(a, '0' + str(n) + 'b')[::-1]
    b_binary = format(b, '0' + str(n) + 'b')[::-1]

    for i in range(n):
        if a_binary[i] == '1':
            circuit.x(q[i])
        if b_binary[i] == '1':
            circuit.x(q[i+n])

    circuit1 = RGQFTMultiplier(num_state_qubits=n, num_result_qubits=2*n)
    circuit = circuit.compose(circuit1)
    for i in range(2*n):
        circuit.measure(q[2*n+i], c[i])

    print(circuit)
    backend = Aer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=100)
    result = job.result()
    counts = result.get_counts()

    #print(counts)
    # Récupérer la multiplication en base 10
    mul_decimal = 0
    for count in counts:
        mul_decimal = int(count[0::], 2)
    return mul_decimal
multiplication(3,3)

     ┌───┐┌──────────────────┐            
q_0: ┤ X ├┤0                 ├────────────
     ├───┤│                  │            
q_1: ┤ X ├┤1                 ├────────────
     ├───┤│                  │            
q_2: ┤ X ├┤2                 ├────────────
     ├───┤│                  │            
q_3: ┤ X ├┤3                 ├────────────
     └───┘│  RGQFTMultiplier │┌─┐         
q_4: ─────┤4                 ├┤M├─────────
          │                  │└╥┘┌─┐      
q_5: ─────┤5                 ├─╫─┤M├──────
          │                  │ ║ └╥┘┌─┐   
q_6: ─────┤6                 ├─╫──╫─┤M├───
          │                  │ ║  ║ └╥┘┌─┐
q_7: ─────┤7                 ├─╫──╫──╫─┤M├
          └──────────────────┘ ║  ║  ║ └╥┘
c: 4/══════════════════════════╩══╩══╩══╩═
                               0  1  2  3 


9

In [16]:
for i in range(11):
    for j in range(11):
        assert multiplication(i,j) == i*j

     ┌──────────────────┐      
q_0: ┤0                 ├──────
     │                  │      
q_1: ┤1                 ├──────
     │  RGQFTMultiplier │┌─┐   
q_2: ┤2                 ├┤M├───
     │                  │└╥┘┌─┐
q_3: ┤3                 ├─╫─┤M├
     └──────────────────┘ ║ └╥┘
c: 2/═════════════════════╩══╩═
                          0  1 
          ┌──────────────────┐      
q_0: ─────┤0                 ├──────
     ┌───┐│                  │      
q_1: ┤ X ├┤1                 ├──────
     └───┘│  RGQFTMultiplier │┌─┐   
q_2: ─────┤2                 ├┤M├───
          │                  │└╥┘┌─┐
q_3: ─────┤3                 ├─╫─┤M├
          └──────────────────┘ ║ └╥┘
c: 2/══════════════════════════╩══╩═
                               0  1 
          ┌──────────────────┐            
q_0: ─────┤0                 ├────────────
          │                  │            
q_1: ─────┤1                 ├────────────
          │                  │            
q_2: ─────┤2              